<a href="https://colab.research.google.com/github/yasserrida/active-learning-based-on-volumes/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from strategies import RandomSampling, LeastConfidence, MarginSampling, EntropySampling, KMeansSampling
from dataset import get_dataset, get_handler
from model import get_net
from torchvision import transforms
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
NB_INITIAL_ETIQUITE = 5000
NB_QUERY = 1000
NB_ITERATIONS = 10
DATASET = 'MNIST'
ARGS_POOL = {'MNIST':
             {'n_epoch': 10, 'transform': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
              'loader_tr_args': {'batch_size': 64, 'num_workers': 1},
              'loader_te_args': {'batch_size': 1000, 'num_workers': 1},
              'optimizer_args': {'lr': 0.01, 'momentum': 0.5}},
             'FashionMNIST':
             {'n_epoch': 10, 'transform': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
              'loader_tr_args': {'batch_size': 64, 'num_workers': 1},
              'loader_te_args': {'batch_size': 1000, 'num_workers': 1},
              'optimizer_args': {'lr': 0.01, 'momentum': 0.5}},
             'SVHN':
             {'n_epoch': 20, 'transform': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
              'loader_tr_args': {'batch_size': 64, 'num_workers': 1},
              'loader_te_args': {'batch_size': 1000, 'num_workers': 1},
              'optimizer_args': {'lr': 0.01, 'momentum': 0.5}},
             }

In [ ]:
# ========== générer un pool étiqueté initial
def label_initial_data():
    index_etiquite = np.zeros(n_pool, dtype=bool)
    index_temp = np.arange(n_pool)
    np.random.shuffle(index_temp)
    index_etiquite[index_temp[:NB_INITIAL_ETIQUITE]] = True
    return index_etiquite

In [ ]:
 # ========= Trainer et tester le modéle
def main_function(strategy, index_etiquite):
    strategy.train()
    P = strategy.predict(x_test, y_test)
    # ========= Calculer la precision Initial
    presions = np.zeros(NB_ITERATIONS + 1)
    presions[0] = 1.0 * (y_test == P).sum().item() / len(y_test)
    print('\nStratégie : ' + type(strategy).__name__)
    print('\tItération 0' + ' ............................. ' +'Précision: {}'.format(presions[0]))
    # ========= Repeter pour le nombre d'iteration
    for rd in range(1, NB_ITERATIONS + 1):
        q_idxs = strategy.query(NB_QUERY)
        index_etiquite[q_idxs] = True
        # ========= mise a jour la dataset
        strategy.update(index_etiquite)
        strategy.train()
        # ========= Calculer la precision
        P = strategy.predict(x_test, y_test)
        presions[rd] = 1.0 * (y_test == P).sum().item() / len(y_test)
        print('\tItération {}'.format(rd) + ' ............................. ' + 'Précision: {}'.format(presions[rd]))
    return presions

In [ ]:
result = []
# ======= Définir le seed
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.enabled = False
# ========== Diviser la dataset
x_train, y_train, x_test, y_test = get_dataset(DATASET)
x_train = x_train[:40000]
y_train = y_train[:40000]
# ======= Affichage des informations
n_pool = len(y_train)
print("\nDataSet : " + DATASET)
print('\tNombre de pool étiqueté: {}'.format(NB_INITIAL_ETIQUITE))
print('\tNombre de pool non étiqueté: {}'.format(n_pool - NB_INITIAL_ETIQUITE))
print('\tNombre de pool de test: {}'.format(len(y_test)))

In [ ]:
# ========= Random Sampling
strategy = RandomSampling(x_train, y_train, label_initial_data(), get_net(DATASET), get_handler(DATASET), ARGS_POOL[DATASET])
result.append(main_function(strategy, label_initial_data()))